# Hello World example for trustpy Python package

This is a simple hello world notebook illustrating what can be done with trustpy.

The TRAD2 has already been generated and is provided in the same folder as this notebook
(if you care, it was generated with an intermediate TRUST version somewhere between 1.9.2 
and 1.9.3).

The example detailed here:
- generates the Python data model from this TRAD2 file -> Python module 'hello_gen.py' is produced.
- and then illustrates how an actual dataset (coherent with the TRAD2 provided) can be loaded, and manipulated. 

To run this notebook properly, you should ensure that the Python environment has been properly loaded
with

```
    source $TRUST_ROOT/env_for_python.sh
```

**Note** : this dataset might not be valid anymore as TRUST versions evolve, but it is coherent 
with the TRAD2 file provided in this folder.


## Setup

We will need the following imports:

In [ ]:
import trustpy.baseTru as BTRU
import xyzpy.classFactoryXyz as CLFX

Let's turn the loggin level a bit down since otherwise the process is a bit verbose:

In [ ]:
import trustpy.trust_utilities as tu
tu._log_level = 1 # Keep only errors (by default, we have warns and infos ...)

## Python data model generation

This step can be skipped if you are working with a stable TRUST version and are not interested in modifying the TRUST grammar (i.e. you are not changing any syntax of a keyword, or not adding a new keyword, a new attribute for a keyword, etc.)

Here we generate the Python data model from the TRAD2 file. This produces a Python module (hello_gen.py) which encodes all the grammar of the TRUST keywords, i.e. what parameter they take, with what type, etc...

In [ ]:
import trustpy.trad2_code_gen as T2C

trad2 = "TRAD2_example"
th = T2C.Trad2Handler(file_in=trad2, file_out="hello_gen.py")
ok = th.trad2xyz()
print("Automatic generation OK:", ok)

## Dataset parsing and manipulation

We are now ready to parse and manipulate our first dataset!

First we import what was generated (again very verbose, but not to worry), and initialize some stuff (warnings can be ignored ...)

In [ ]:
import hello_gen as hg  # you can replace this with the official TRUST generated module if you didn't take previous step

BTRU.initXyzClasses()
hg.initXyzClasses()    

Now we invoke the parser to load a TRUST dataset. The parser splits the dataset (a .data file) to produce a stream of tokens (=a somewhat clever list of strings, if you want). This stream will be next passed to the data model constructor.

The parser (and the associated stream it produces) deals with the comments and other formatting elements.

In [ ]:
from trustpy.trust_parser import TRUSTParser, TRUSTStream

fNam = "upwind_simplified.data"
with open(fNam) as f:
    data_ex = f.read()

tp = TRUSTParser()
tp.tokenize(data_ex)
stream = TRUSTStream(tp)

In [ ]:
print(stream)

Let's print the dataset for future reference:

In [ ]:
print(data_ex)

And finally we load this into the data model that we have generated above:

In [ ]:
# trustpy (and pacakgepy) use a class factory mechanism. We retrieve the top level class 'DataSet_Tru' and 
# invoke the main method 'ReadFromTokens()' which will load the data into the model.
# If you don't understand what I just said, consider this as black magic that you can copy/paste everytime :-)
tds_cls = CLFX.getXyzClassFromName("DataSet_Tru")
ds = tds_cls.ReadFromTokens(stream)

Now **ds** contains our TRUST dataset as a Python object and we can finally play with it! We show several examples:

### Example: reading/adding a time scheme attribute

What is the current final time in our dataset ? Note in the below how the Python attributes exactly corresponds to the name of the attribute in the TRUST syntax.

In [ ]:
# Named object in the dataset are retrieved using the get() method:
ze_time_scheme = ds.get("sch")

# Then attributes can be read using the option name of the TRUST keyword, here 'tmax':
t_mx = ze_time_scheme.tmax
print("tmax is %g" % t_mx )

If we want to inspect the list of available attributes, we can do:

In [ ]:
print(ze_time_scheme._attributesList)

Another (clearer?) possibility is:

In [ ]:
from pprint import pprint  # Useful Python pretty print
pprint(ze_time_scheme._helpDict)

We can now set an extra parameter:

In [ ]:
ze_time_scheme.facsec = 1.3

### Example: reading the name of the first probe
Same story with a more complex things: the name of the first probe ...

In [ ]:
# Retrieve the problem named 'pb' in the dataset:
ze_pb = ds.get("pb")

# Read the first probe in the postprocessing block:
first_probe = ze_pb.post_processing.probes[0]
print(first_probe.nom_sonde)

... and then the x coordinate of the first point in the first probe: notice how some attributes might be painful and very dependent on how the TRUST grammar was built. In this case for example, the somewhat cryptic 'type' intermediate argument comes from the way the TRAD2 is defined.

In [ ]:
y = first_probe.type.points[0].pos[0]
print(y)

## Writing back the dataset
The data can be modified ... and written back in a form of a new dataset!

Let's modify the y coordinate of the first probe:

In [ ]:
first_probe.type.points[0].pos[0] = -42.1  # as simple as that!

Note that if you try to input an invalid type, the module will complain - try this for example:

In [ ]:
# first_probe.type.points[0].pos[0] = "some text"  # will fail

We can also for example delete the second probe:

In [ ]:
del(ze_pb.post_processing.probes[1])

Now we produce a new stream of tokens corresponding to this modified instance. This stream of tokens can then be simply joined to find back a textual dataset with the modifed data. Notice how all the other comments and formatting is kept unchanged.

In [ ]:
# Convert back the data into a stream of tokens:
newStream = ds.toDatasetTokens() 
# And write it out!
s = ''.join(ds.toDatasetTokens())
print(s)